In [9]:
from H20_Nakata_model import *

In [10]:
import datetime

import numpy as np
import pandas as pd
import pandas.io.data as web
import quandl


def setdata(s, e, t='Adj Close'):
    start = datetime.datetime(s[0], s[1], s[2])
    end = datetime.datetime(e[0], e[1], e[2])
    N225 = web.DataReader('^N225', 'yahoo', start, end)[t]
    DJI = web.DataReader('^DJI', 'yahoo', start, end)[t]
    IXIC = web.DataReader('^IXIC', 'yahoo', start, end)[t]
    jpy = web.DataReader('DEXJPUS', 'fred')
    jpy = jpy.ix[start:end]
    usdjpy = quandl.get("BOE/XUDLJYD", start_date=start, end_date=end).assign(
                    USDJPY_roc = lambda df: df["Value"].pct_change()
                ).rename(columns={
                    "Value" : "USDJPY"
                })
    eurjpy = quandl.get("BOE/XUDLBK63", start_date=start, end_date=end).assign(
                    EURJPY_roc = lambda df: df["Value"].pct_change()
                ).rename(columns={
                    "Value" : "EURJPY"
                })
    gbpjpy = quandl.get("BOE/XUDLJYS", start_date=start, end_date=end).assign(
                    EURJPY_roc = lambda df: df["Value"].pct_change()
                ).rename(columns={
                    "Value" : "GBPJPY"
                })
    usdjpy = usdjpy["USDJPY"]
    eurjpy = eurjpy["EURJPY"]
    gbpjpy = gbpjpy["GBPJPY"]
    
    dataframe = pd.concat([N225, DJI, IXIC, usdjpy, eurjpy, gbpjpy], axis=1).dropna()
    
    datas = np.array(dataframe)
    target = datas.T[0]

    return datas, target

In [11]:
data, target = setdata((2002, 10, 1), (2006, 4, 1))
data

array([[ 11723.629883,  10404.299805,   1984.810059,    107.38    ,
           139.18    ,    202.8193  ],
       [ 11667.540039,  10421.139648,   1991.069946,    108.33    ,
           138.94    ,    202.8154  ],
       [ 11774.30957 ,  10458.459961,   1999.319946,    108.5     ,
           139.29    ,    203.6328  ],
       ..., 
       [ 16938.410156,  11215.700195,   2337.780029,    118.1     ,
           141.72    ,    204.75    ],
       [ 17045.339844,  11150.700195,   2340.820068,    117.22    ,
           142.35    ,    204.631   ],
       [ 17059.660156,  11109.320312,   2339.790039,    117.97    ,
           142.77    ,    204.619   ]])

In [13]:
for i in range(len(data.T)):
    data.T[i] = (data.T[i] - np.average(data.T[i]))/np.max(abs(data.T[i] - np.average(data.T[i])))

target = (target - np.average(target))/np.max(abs(target - np.average(target)))

日経平均とその他のデータとの相関。

In [14]:
print(np.corrcoef(data.T[0], data.T[1])[0, 1])
print(np.corrcoef(data.T[0], data.T[2])[0, 1])
print(np.corrcoef(data.T[0], data.T[3])[0, 1])
print(np.corrcoef(data.T[0], data.T[4])[0, 1])
print(np.corrcoef(data.T[0], data.T[5])[0, 1])

0.815115223996
0.866370655774
0.872475524899
0.766890210222
0.722850448569


In [15]:
pred = data[-1]
pred

array([ 1.        ,  0.6937186 ,  0.7103626 ,  0.55246872,  0.73558563,
        0.2457783 ])

訓練用のデータとそのラベルの作成

In [16]:
case_A, case_B, case_C, case_D = make_input(data[:-1])
label = make_label(target)

In [17]:
y_A = label[1:]
y_B = label[5:]
y_C = label[5:]
y_D = label[5:]

テスト用のデータとそのラベルの作成

In [18]:
test_data, test_target = setdata((2007, 1, 1), (2007, 12, 1))

In [19]:
for i in range(len(data.T)):
    test_data.T[i] = (test_data.T[i] - np.average(test_data.T[i]))/np.max(abs(test_data.T[i] - np.average(test_data.T[i])))

test_target = (test_target - np.average(test_target))/np.max(abs(test_target - np.average(test_target)))

In [20]:
test_case_A, test_case_B, test_case_C, test_case_D = make_input(test_data[:-1])

In [21]:
test_label = make_label(test_target)
test_label_A = test_label[1:]
test_label_B = test_label[5:]
test_label_C = test_label[5:]
test_label_D = test_label[5:]

In [22]:
P_A = PredictDirection_PriceChanges(case_A, y_A, pred)
P_A.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0

(0.044026319064537987, 19.026605745618902, 0.6782608695652174)

In [23]:
P_A.descript(test_case_A, test_label_A)

lnσ^2= -3.12296766333
lnC= 2.94583830206
識別誤り率は 0.32173913043478264
識別成功率は 0.616822429906542
[[ 98.  66.]
 [ 16.  34.]]
グリッドサーチ最高点での識別成功率は 0.616822429906542


0.616822429906542

In [24]:
P_B = PredictDirection_PriceChanges(case_B, y_B, pred)
P_B.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0

(0.35241341267226833, 17226.616511806002, 0.6304347826086957)

In [26]:
P_B.descript(test_case_B, test_label_B)

lnσ^2= -1.04295032471
lnC= 9.75421093832
識別誤り率は 0.3695652173913043
識別成功率は 0.6190476190476191
[[ 85.  53.]
 [ 27.  45.]]
グリッドサーチ最高点での識別成功率は 0.6190476190476191


0.6190476190476191

In [27]:
P_C = PredictDirection_PriceChanges(case_C, y_C, pred)
P_C.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0

(0.15200763017357058, 699.92403898385123, 0.6217391304347826)

In [28]:
P_C.descript(test_case_C, test_label_C)

lnσ^2= -1.88382456089
lnC= 6.55097181342
識別誤り率は 0.37826086956521743
識別成功率は 0.6095238095238096
[[ 84.  54.]
 [ 28.  44.]]
グリッドサーチ最高点での識別成功率は 0.6190476190476191


0.6095238095238096

In [29]:
P_D = PredictDirection_PriceChanges(case_D, y_D, pred)
P_D.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0

(0.070789866424609021, 0.44742997598041445, 0.591304347826087)

In [30]:
P_D.descript(test_case_D, test_label_D)

lnσ^2= -2.64803941812
lnC= -0.804235231767
識別誤り率は 0.408695652173913
識別成功率は 0.5714285714285714
[[ 107.   85.]
 [   5.   13.]]
グリッドサーチ最高点での識別成功率は 0.5666666666666667


0.5714285714285714

In [31]:
from sklearn.metrics import confusion_matrix
print confusion_matrix(label_test, label_predict)

SyntaxError: invalid syntax (<ipython-input-31-5f6b98d6f9cb>, line 2)

In [35]:
from sklearn import svm, grid_search, datasets
parameters = {'kernel':('rbf'), 'C':[1, 10]}
svr = svm.SVC()
clf = grid_search.GridSearchCV(svr, parameters)
clf.fit(case_C, y_C)

ValueError: Parameter values should be a list.